# imports

In [ ]:
import os

In [ ]:
import cv2

In [ ]:
import random

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

# config

In [ ]:
#原始文件的存放路径
g_srcimg_ori_path = './data/src_img/initial'

In [ ]:
#从标记的图片找到的矩形的记录
g_boxes_csv_pathfn = './data/src_img/allboxes.csv'

In [ ]:
# 生成的数据集的存放路径
g_ds_root = './data/ds_20200426/'

In [ ]:
# 生成的数据集的csv文件名
g_ds_csv_fn = 'gends.csv'
ds_csv_path = g_ds_root + g_ds_csv_fn

In [ ]:
# 生成的数据集的图片文件夹名
g_ds_img_subdir = 'images'
ds_img_path = g_ds_root + g_ds_img_subdir

# 函数

In [ ]:
def chk_params(data_root_path, boxes_csv_fn, srcimg_ori_path, output_subimg_path, output_ds_csv_fn, params, dbg):
    """
    功能：    
        1.检查一下参数是否有效。
        2.单层子目录和文件名转换成带目录的完整目录名。 
        3.检查调用方传入的params，空缺的值用默认参数补齐。
    参数：
        data_root_path：存放数据的根目录。
        boxes_csv_fn：存放图片符号信息的csv文件。由find_boxes生成。
        srcimg_ori_path：存放未标记的原始大图的目录。
        output_subimg_path：存放生成的数据集图片的子目录名，在根目录下。
        output_ds_csv_fn：存放生成的数据集的csv文件名，在根目录下。
        params：调用方指定的控制参数。
        dbg：调试信息。
    返回值：
        1.None/错误信息，成功返回None，否则是错误信息。
        2.存放图片的目录的完整目录名
        3.生成的csv的完整文件名
        4.最用使用控制参数dict。
    """
    
    #控制生成的参数
    PM = {}
    
    if '' == data_root_path: 
        return "invalid data_root_path", data_root_path, boxes_csv_fn, srcimg_ori_path, output_subimg_path, output_ds_csv_fn, PM
    
    if '' == output_ds_csv_fn or output_ds_csv_fn.find('/') >= 0 or output_ds_csv_fn.find('\\') >= 0: 
        return 'invalid output_ds_csv_fn', data_root_path, boxes_csv_fn, srcimg_ori_path, output_subimg_path, output_ds_csv_fn, PM

    if '' == boxes_csv_fn: 
        return 'invalid boxes_csv_fn', data_root_path, boxes_csv_fn, srcimg_ori_path, output_subimg_path, output_ds_csv_fn, PM
    
    if '' == srcimg_ori_path: 
        return 'invalid srcimg_ori_path', data_root_path, boxes_csv_fn, srcimg_ori_path, output_subimg_path, output_ds_csv_fn, PM 
        
    if output_subimg_path.find('/') >= 0 or output_subimg_path.find('\\') >= 0: 
        return 'invalid srcimg_ori_path', data_root_path, boxes_csv_fn, srcimg_ori_path, output_subimg_path, output_ds_csv_fn, PM 
        
    if data_root_path[-1] not in ['/', '\\']:
        data_root_path += '/'
        
    output_subimg_path = os.path.join(data_root_path, output_subimg_path)
    output_ds_csv_fn = os.path.join(data_root_path, output_ds_csv_fn)
         
    if not os.path.exists(output_subimg_path):
        os.makedirs(output_subimg_path)
    #print('making dirs:', subimg_path, )
     
    #global DSSIZE, BLANKP, DSIMGW, DSIMGH, ZOOM, ZOOMSP, WHITE_TH 
    default_params = {
        #总的数据集的数量
        'DSSIZE': 10000,
        #没有符号的占比
        'BLANKP':  0.25,
        #数据集图片大小
        'DSIMGW': 416,
        'DSIMGH': 416,
        #如果有缩放，缩放的范围
        'ZOOM': [0.9, 1.1],
        #多大的比例让两个方向缩放不一致。小于这个值的时候会不一样的时候会不一致。
        'ZOOMSP': 0.25,
        #如果黑点数量比例小于这个值就放弃掉这个图片。
        'WHITE_TH': 20
    }
    for param in ['DSSIZE', 'BLANKP', 'DSIMGW', 'DSIMGH', 'ZOOM', 'ZOOMSP', 'WHITE_TH']:
        #dcode = '%s' % param
        #if 'dbg' in param:
        #    print('>>>', dcode, eval(dcode, globals()))
        #PM[param] = eval(dcode) 
        if param not in params:
            PM[param] = default_params[param] 
        else:
            PM[param] = params[param]
     
    if 'dbg' in dbg:
        print('PM', PM)
     
    return None, output_subimg_path, output_ds_csv_fn, PM

In [ ]:
def gen_datasets(data_root_path, boxes_csv_fn = './data/allboxes.csv', srcimg_ori_path = './data/src_img/initial'
                 , output_subimg_path = 'images', output_ds_csv_fn = 'gends.csv'
                 , params = {
                    #总的数据集的数量
                    'DSSIZE': 10000,
                    #没有符号的占比
                    'BLANKP': 0.25,
                    #数据集图片大小
                    'DSIMGW': 416,
                    'DSIMGH': 416,
                    #缩放的范围
                    'ZOOM': [0.9, 1.1],
                    #多大的比例让两个方向缩放不一致。小于这个值的时候会不一样的时候会不一致。
                    'ZOOMSP': 0.25,
                    #如果黑点数量小于这个值就作为全白的图片放弃掉。
                    'WHITE_TH': 20
                 }
                 , dbg = []):
    """
    生成数据集
    参数：        
        data_root_path存放数据的根目录名。不能为空
        boxes_csv_fn：存放图片符号信息的csv文件。由find_boxes生成。不能为空。
        srcimg_ori_path：存放未标记的原始大图的目录。不能为空
        output_subimg_path：存放生成的数据集图片的子目录名，在根目录下。只能有一层。
        output_ds_csv_fn：存放生成的数据集描述信息的csv文件名，在根目录下。默认gends.csv.不能为空。不能带路径名。
        param:控制生成的参数。
        #dbg:调试信息 
    返回值:
        1.成功True/失败False
        2.错误信息（失败时）/数据集的描述信息（成功时）
            数据集描述信息：
            {
                image: 生成的图片文件名list。
                box: list，每个元素与image一一对应，是图片中包含的符号的位置信息的box列表。
                cls：list，每个元素与image一一对应，是图片中包含的符号的类型(字符串)列表。
                tag：list，每个元素与image一一对应，是图片中包含的符号在原始大图上的位置信息列表。
            }
        
    """
    #记录下存放图片的子目录的名字,放到文件名里面。
    sub_path_raw = output_subimg_path 
    
    err, output_subimg_path, output_ds_csv_fn, PM = chk_params(data_root_path
                            , boxes_csv_fn, srcimg_ori_path, output_subimg_path, output_ds_csv_fn, params, dbg)  
    #print('pm', PM)
    if err is not None: 
        return False, err
    
    df = pd.read_csv(boxes_csv_fn)
    
    cnts = {} 
    totalcoms = 0
    imgs = df.loc[:, 'image'] 
    boxes = df.loc[:, 'box']
    classes = df.loc[:, 'cls']
    
    #简单统计一下，每张大图里面包含的符号的数量。用于按比例生成相应数量图片。符号越多在数据集中生成的小图也越多。
    for i in range(len(imgs)):
        img = imgs[i]
        cnts[img] = {}
        #print(len(eval(boxes[i])))
        tbox = eval(boxes[i])
        box = []
        #转换一下格式!!!!!!!!!!!!!这个原始的格式是y和x是反的，是numpy的格式
        for b in tbox:
            box.append([b[1], b[0], b[3], b[2]])
        clas = eval(classes[i])
        cnts[img]['comscnt'] = len(box)
        cnts[img]['allbox'] = box
        cnts[img]['allcom'] = clas
        #记录一下具体的每种符号的数量和位置
        cnts[img]['typscnt'] = {}
        for j in range(len(clas)):
            cls = clas[j]
            if cls not in cnts[img]['typscnt']:
                cnts[img]['typscnt'][cls] = {'cnt': 0, 'boxes': []}
            cnts[img]['typscnt'][cls]['cnt'] = cnts[img]['typscnt'][cls]['cnt'] + 1
            cnts[img]['typscnt'][cls]['boxes'].append(box[j])
            #if cls not in typscnt:
            #    typscnt[cls] = 0
            #typscnt[cls] += 1
            totalcoms += 1
     
    if 'prt_info' in dbg:
        print('totalcoms:', totalcoms)
    
    #开始生成 
    datasets = {}#记录大图片的里面提取小图的时候的一些信息，位置，大小等。
    
    #下面这个是最终的结果,小图片的信息
    ds = {'image':[], 'box': [], 'cls': [], 'tag': []}
    
   
    for i in range(len(imgs)): 
        imgfn = imgs[i]
        #跳过没有标记的。可能会导致生成的空白图里面包含符号
        if cnts[imgfn]['comscnt'] == 0:
            if 'dbg' in dbg:
                print('ignore:', imgfn)
            continue
            
        #datasets[imgfn] = []
        #当前图片应该产生产生的数据集的数量。总数，带有符号的有效的图片，没有符号的空白
        total = int(cnts[imgfn]['comscnt'] / totalcoms * PM['DSSIZE'])
        blankc = int(PM['BLANKP'] * total)
        validc = total - blankc
        if 'prt_info' in dbg:
            print(imgfn, total, blankc, validc)
        
        img = cv2.imdecode(np.fromfile(os.path.join(srcimg_ori_path, imgfn), dtype=np.uint8), -1)
        imgh, imgw, _ = img.shape
        
        #生成的数量统计
        blanks = 0
        valids = 0
        trycnt = 0
        while True:
            #还是没有足够的数据，放弃
            if trycnt > PM['DSSIZE']:
                break
            #数量够了    
            if valids > validc and blanks > blankc:
                break
            trycnt += 1
            #随机一个起始点坐标。
            x = random.randint(0, imgw - int(PM['DSIMGW'] * PM['ZOOM'][1]))
            y = random.randint(0, imgh - int(PM['DSIMGH'] * PM['ZOOM'][1])) 
            #tozoom = random.random() < PM['IFZOOM']
            tozoom = True
            fz = 1.0
            if tozoom:
                fz = random.uniform(PM['ZOOM'][0], PM['ZOOM'][1])#缩放
            W = int(PM['DSIMGW'] * fz)
            #y方向的缩放产生点不一致
            if tozoom and random.random() < PM['ZOOMSP']:
                fz = random.uniform(PM['ZOOM'][0], PM['ZOOM'][1])
                
            H = int(PM['DSIMGH'] * fz) 
             
            simgfn = imgfn + '_s_%d_%d_%d_%d__%d_%d_%d.jpg' % (x, y, x + W, y + H, W, H, len(datasets))
            tag = (imgfn, x, y, x + W, y + H, W, H, len(datasets))
            if simgfn in datasets:
                continue 
            datasets[simgfn] = []
            #看一下小图里面是否有符号
            isvalid = False
            for k in range(len(cnts[imgfn]['allbox'])):
                box = cnts[imgfn]['allbox'][k]
                cx = x + W // 2
                cy = y + H // 2
                #符号坐标
                bx = (box[0] + box[2]) // 2
                by = (box[1] + box[3]) // 2
                bw = box[2] - box[0]
                bh = box[3] - box[1]
                #符号尺寸肯定小雨图片尺寸
                #一个符号完整的存在
                if abs(cx - bx) <= (W - bw) // 2 and abs(cy - by) <= (H - bh) // 2:
                    #发现了符号了。如果有符号的数据集数量不够就搞一个。
                    if valids <= validc:
                        isvalid = True
                        item = {'com': cnts[imgfn]['allcom'][k], 'box': box, 's': 'v'} 
                        datasets[simgfn].append(item)
                        
                    else:#有效的数据集已经够了。记录下来后面抹掉符号。作为空白数据集用
                        item = {'com': cnts[imgfn]['allcom'][k], 'box': box, 's': 'b'} 
                        datasets[simgfn].append(item) 
                #包含了一部分符号
                elif abs(cx - bx) < (W + bw) // 2 and abs(cy - by) < (H + bh) // 2: 
                    item = {'com': cnts[imgfn]['allcom'][k], 'box': box, 's': 'b'} 
                    datasets[simgfn].append(item) 
                else:#完全空白的。可以直接使用的
                    pass
             
            if isvalid:
                valids += 1
            else:
                if blanks > blankc:#空白（没有符号）的数量够了。跳过循环
                    continue
                blanks += 1
           
            simg = img.copy()[y : y + H, x : x + W]
            simg2 = simg.copy()
            simg3 = cv2.resize(simg.copy(), (PM['DSIMGW'], PM['DSIMGH']))#标记图片，转换到标准大小的。
            #缩放后转回到标准大小的比例
            rx, ry = PM['DSIMGW'] / W, PM['DSIMGH'] / H
                 
            boxes = []
            cls = []
            for it in datasets[simgfn]:
                if 'prt_info' in dbg: 
                    print('1ds', simgfn, it)
                box = it['box']
                if it['s'] == 'v':#包含完整符号，记录下来
                    if 'mark' in dbg:
                        cv2.rectangle(simg, (box[0] - x, box[1] - y), (box[2] - x, box[3] - y), (0, 0, 255), 4)
                        cv2.rectangle(simg3, (int((box[0] - x) * rx), int((box[1] - y) * ry))
                                           , (int((box[2] - x) * rx), int((box[3] - y) * ry)), (0, 0, 255), 4)
                    #转换到numpy的顺序，先y后x
                    #!!!boxes.append([box[1] - y, box[0] - x, box[3] - y, box[2] - x])
                    boxes.append([int((box[1] - y) * ry), int((box[0] - x) * rx), int((box[3] - y) * ry), int((box[2] - x) * rx)])
                    
                    cls.append(it['com'])
                    #pass
                else: #把其余不完整符号都抹掉
                    #print('rectangle', box, x, y) 
                    if 'mark' in dbg:
                        cv2.rectangle(simg, (box[0] - x, box[1] - y), (box[2] - x, box[3] - y), (255, 0, 0), 4)
                        cv2.rectangle(simg3, (int((box[0] - x) * rx), int((box[1] - y) * ry))
                                           , (int((box[2] - x) * rx), int((box[3] - y) * ry)), (255, 0, 0), 4)
                    #抹掉不完整符号的时候可能会有离得比较近的符号被抹掉一小部分。先不管它
                    cv2.rectangle(simg2, (box[0] - x, box[1] - y), (box[2] - x, box[3] - y), (255, 255, 255), -1)
            
            #不包含符号。看一下是不是完全空白的
            if not isvalid:
                gray = cv2.cvtColor(simg2, cv2.COLOR_BGR2GRAY)
                thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
                dcnt = cv2.countNonZero(thresh)
                if 'prt_info' in dbg:        
                    print('countnonzero:', dcnt, W * H)                
                
                if dcnt > W * H - PM['WHITE_TH']:
                    if 'prt_info' in dbg:
                        print('白板1!!!!!')
                    blanks -= 1
                    continue
                
            #cv2.imencode('.jpg', simg)[1].tofile('datasets/mark_' + simgfn)         
            #cv2.imencode('.jpg', simg2)[1].tofile('datasets/' + simgfn)      
            nfn = '%05d.jpg' % len(ds['image'])
            if 'mark' in dbg:
                cv2.imencode('.jpg', simg)[1].tofile(os.path.join(output_subimg_path, 'mark_' + nfn))
                cv2.imencode('.jpg', simg3)[1].tofile(os.path.join(output_subimg_path, 'mark2_' + nfn))
            
            #把数据集里面的图片改回到标准尺寸。不用训练过程中专门每次缩放
            if W != PM['DSIMGW'] or H != PM['DSIMGH']:
                simg2 = cv2.resize(simg2, (PM['DSIMGW'], PM['DSIMGH']))                
            cv2.imencode('.jpg', simg2)[1].tofile(os.path.join(output_subimg_path, nfn))
            
            ds['image'].append(sub_path_raw + '/' + nfn)
            ds['box'].append(boxes)
            ds['cls'].append(cls)
            ds['tag'].append(tag)
            
             
        if 'prt_info' in dbg:
            print(valids, blanks, validc, blankc)        
        #break
    #print(datasets)
    df = pd.DataFrame(ds)
    df.to_csv(output_ds_csv_fn)  
    return True, ds
     

# 生成

In [ ]:
#默认参数生成
gen_datasets(data_root_path = g_ds_root, 
             boxes_csv_fn = g_boxes_csv_pathfn, 
             srcimg_ori_path = g_srcimg_ori_path, 
             output_subimg_path = g_ds_img_subdir, 
             output_ds_csv_fn = g_ds_csv_fn,
             params = {'DSSIZE': 10000,
                       'BLANKP': 0.1,
                       'DSIMGW': 776,
                       'DSIMGH': 776,
                       'ZOOM': [0.5, 2],
                       'ZOOMSP': 0,
                       'WHITE_TH': 20
                       }
             )